In [1]:
import tskit
import msprime
import pyslim

# Recapitation: tying up loose ends

First, we run the [SLiM simulation](recap_example.slim), to generate the tree sequence.

In [2]:
# Running a SLiM simulation
! slim recap_example.slim

// Initial random seed:
3811278872788783660

// RunInitializeCallbacks():
initializeSLiMModelType(modelType = 'WF');
initializeTreeSeq();
initializeMutationRate(1e-08);
initializeMutationType(1, 0, "f", 0);
initializeGenomicElementType(1, m1, 1);
initializeGenomicElement(g1, 0, 99999);
initializeRecombinationRate(1e-08);
#WARNING (Species::RunInitializeCallbacks): with tree-sequence recording enabled and a non-zero mutation rate, a neutral mutation type was defined and used; this is legal, but usually undesirable, since neutral mutations can be overlaid later using the tree-sequence information.

// Starting run at tick <start>:
1 



We can extract useful information on the command line with the `tskit` command;
for instance, printing out the populations:

In [3]:
! tskit populations recap_example.trees

id	metadata
0	None
1	{'migration_records': [], 'name': 'p1', 'sex_ratio': 0.0, 'slim_id': 1}
2	{'migration_records': [], 'name': 'p2', 'sex_ratio': 0.0, 'slim_id': 2}


Now, in python, we recapitate.
The warning is helpful, since it reminds us that the time units of the SLiM simulation
might not be in generations.
However, this was a Wright-Fisher simulation, and we are providing a recombination rate
in units of recombinations per bp per generation, so we're good.
See [the pyslim docs](https://tskit.dev/pyslim/docs/latest/time_units.html) for more on this topic.

In [4]:
ts = tskit.load("recap_example.trees")
demography = msprime.Demography.from_tree_sequence(ts, initial_size=100)

demography.add_migration_rate_change(
    time=ts.metadata["SLiM"]["tick"] + 100,
    rate=0.1, source="p2", dest="p1")
rts = pyslim.recapitate(ts, demography=demography,
            recombination_rate=1e-8)

/home/peter/projects/tskit-dev/tsvenv/lib/python3.13/site-packages/msprime/ancestry.py:1290: TimeUnitsMismatchWarning: The initial_state has time_units=ticks but time is measured in generations in msprime. This may lead to significant discrepancies between the timescales. If you wish to suppress this warning, you can use, e.g., warnings.simplefilter('ignore', msprime.TimeUnitsMismatchWarning)
  sim = _parse_sim_ancestry(


In [5]:
rts